# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, learning_curve,\
train_test_split, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector

## pipeline stuff

from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn import set_config; set_config(display='diagram')

import warnings 
warnings.filterwarnings('ignore')

from sklearn.metrics import make_scorer
from tqdm.auto import tqdm


# Preprocessing

In [2]:
# def_df = pd.read_csv("/home/slawa/code/code-rep0/projects/data/defaulter_data_13364.csv", index_col=[0])
# pay_df = pd.read_csv("/home/slawa/code/code-rep0/projects/data/payer_data_41940.csv", index_col=[0])

def_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_6k_ids_compress.parquet")
# def_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_20k_ids_compress.parquet")
pay_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.parquet")

def_df['default'] = 1
pay_df['default'] = 0

df = pd.concat([def_df, pay_df])

df.reset_index(inplace=True)

y = df['default'].reset_index(drop=True)

X = df.drop(columns=['default']).reset_index(drop=True)

cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']

X_corr = X.corr()

X_corr = X_corr.unstack().reset_index() # Unstack correlation matrix 
X_corr.columns = ['feature_1','feature_2', 'correlation_all'] # rename columns
X_corr.sort_values(by="correlation_all",ascending=False, inplace=True) # sort by correlation
X_corr = X_corr[X_corr['feature_1'] != X_corr['feature_2']] # Remove self correlation
X_corr = X_corr.drop_duplicates(subset='correlation_all')

red_features = list(X_corr[abs(X_corr['correlation_all'])>=.95]['feature_1']) ## abs so we also consider the negative corrs

X_red = X.drop(columns=red_features) ## dropping the highly correlated columns

## checking whether the high correlations are gone
X_red_corr = X_red.corr()
X_red_corr = X_red_corr.unstack().reset_index() # Unstack correlation matrix 
X_red_corr.columns = ['feature_1','feature_2', 'correlation_all'] # rename columns
X_red_corr.sort_values(by="correlation_all",ascending=False, inplace=True) # sort by correlation
X_red_corr = X_red_corr[X_red_corr['feature_1'] != X_red_corr['feature_2']] # Remove self correlation
X_red_corr = X_red_corr.drop_duplicates(subset='correlation_all')


## drop columns with nans if in both groups > 80% nans

nan_threshold= 0.8 ## adjust the hardcoded values
def_nans = def_df.isna().sum()/len(def_df) 
def_nans_80 = def_nans[def_nans >= nan_threshold].index
pay_nans = pay_df.isna().sum()/len(pay_df)
pay_nans_80 = pay_nans[pay_nans>= nan_threshold].index
nans_80 = [feature for feature in pay_nans_80 if feature in def_nans_80]

## check whether features were already removed
red_features_nan = [feature for feature in nans_80 if feature not in red_features] 
X_red = X_red.drop(columns=red_features_nan)
dropped_columns = red_features + red_features_nan

## Builsing the pipeline
num_vars = [feature for feature in X_red.columns[2:] if feature not in cat_vars] ## exclude dates and IDs (first two columns)
str_vars = [feature for feature in X_red.columns[2:] if not pd.api.types.is_numeric_dtype(X_red[feature])] ## columns that are not numeric at all 
#red_cat_vars = [feature for feature in cat_vars if feature not in dropped_columns + str_vars] ## remaining categorical variables that have no string values
red_cat_vars = [feature for feature in cat_vars if feature not in dropped_columns + str_vars] ## remaining categorical variables 


# impute mean/most frequent value for other nans (specific to group?)
# robustscale all numerical values

num_imputer = SimpleImputer(strategy='mean')
num_scaler = RobustScaler()

#num_imputer = KNNImputer(n_neighbors=2) ## KNNIMputer is computationally demanding
## should come AFTER SCALING

num_pipe = make_pipeline(num_imputer, num_scaler)

#str_trans = OrdinalEncoder() # is only needed if one wants to do knnimputer

#nan_trans = FunctionTransformer(nan_imp)

#nan_trans = FunctionTransformer(lambda X: X.applymap(lambda x: np.nan if x in [-1,-1.0, "-1.0", "-1"] else x))

cat_imputer = SimpleImputer(strategy="most_frequent") ## replace with KNNimputer on one neighbour, after transforming to numericals
#cat_imputer = KNNImputer(n_neighbors=1) # introducing it did not improve performance, but is computationally demanding
cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore') ## what happens to the old columns?
cat_pipe = make_pipeline(cat_imputer, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, str_trans, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, cat_encoder)
# impute mean/most frequent value for other nans (specific to group?)
# robustscale all numerical values

num_imputer = SimpleImputer(strategy='mean')
num_scaler = RobustScaler()

#num_imputer = KNNImputer(n_neighbors=2) ## KNNIMputer is computationally demanding
## should come AFTER SCALING

num_pipe = make_pipeline(num_imputer, num_scaler)

#str_trans = OrdinalEncoder() # is only needed if one wants to do knnimputer

#nan_trans = FunctionTransformer(nan_imp)

#nan_trans = FunctionTransformer(lambda X: X.applymap(lambda x: np.nan if x in [-1,-1.0, "-1.0", "-1"] else x))

cat_imputer = SimpleImputer(strategy="most_frequent") ## replace with KNNimputer on one neighbour, after transforming to numericals
#cat_imputer = KNNImputer(n_neighbors=1) # introducing it did not improve performance, but is computationally demanding
cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore') ## what happens to the old columns?
cat_pipe = make_pipeline(cat_imputer, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, str_trans, cat_encoder)
#str_pipe = make_pipeline(cat_imputer, cat_encoder)

print('done ✅')


done ✅


In [3]:
print(def_df.shape , pay_df.shape)

(76747, 190) (243953, 190)


In [ ]:
# 6k defaulters = (76747, 190) 20k payer = (243953, 190)  
# 20k defaulters = (116027, 190) 20k payer = (243953, 190)  


# A self-contained alternative Nans imputer

In [4]:
def alt_nan_imp(X):
    
    cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']
    
    alt_nan_list = [-1,-1.0, "-1.0", "-1"]
    
    cat_columns = [column for column in X.columns if column in cat_vars]
    
    X[cat_columns] = X[cat_columns].applymap(lambda x: np.nan if x in alt_nan_list else x)

alt_nan_imp(X_red)

preprocessor = ColumnTransformer([
    ('num_pip', num_pipe, num_vars),
    ('cat_pip', cat_pipe, red_cat_vars)],
    remainder='drop' ## all columns not in num_vars and red_cat_vars are dropped.
)

alt_nan_imp(X)
preprocessor.fit(X)

X_pp = pd.DataFrame(preprocessor.fit_transform(X_red))
X_pp['customer_ID'] = X_red['customer_ID']
X_pp_avg = X_pp.groupby('customer_ID').mean()
y_ID = pd.DataFrame(y)
y_ID['customer_ID'] = X_red['customer_ID']
y_unique = y_ID.groupby('customer_ID').mean().astype(int) ## actually, this data is just in train_labels
X_pp_avg_train, X_pp_avg_val, y_unique_train, y_unique_val = train_test_split(X_pp_avg, y_unique, test_size=0.3) 


if 'customer_ID' in X_pp.columns:
    X_pp.drop(columns='customer_ID', inplace=True)
else:
    pass

# # Train_test_split needs to be on preprocessed data
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)


# pp_pred_pipe = make_pipeline(preprocessor, mod)
# pp_pred_pipe
# pp_pred_pipe.fit(X, y)

print('done ✅')


done ✅


# Creating custom amex scoring metric

In [5]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    
    ## TWEAK
    y_true = pd.DataFrame(y_true.reset_index(drop=True))
    y_pred = pd.DataFrame(y_pred)
    
    y_true = y_true.rename(columns={y_true.columns[0]:'target'})
    y_pred = y_pred.rename(columns={y_pred.columns[0]:'prediction'})
    ##
    
    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

print('done ✅')


done ✅


In [6]:
amex_metric_scorer = make_scorer(amex_metric)

## dict of scoring metrics one might want to pass into cross validation
scorings = {'recall':'recall',
            'f1':'f1',
           'amex': amex_metric_scorer}

print('done ✅')

done ✅


# Averaging preprocessed X and y

In [7]:
X_pp['customer_ID'] = X_red['customer_ID']
# X_avg_pp = X_pp.groupby('customer_ID').mean()
X_avg_pp = X_pp.groupby('customer_ID').agg(['mean', 
                                            'last', 
                                            'first',
                                           'max',
                                           'min'
                                           ])


y_ID = pd.DataFrame(y)
y_ID['customer_ID'] = X_red['customer_ID']
## actually, this data is just in train_labels
y_unique = y_ID.groupby('customer_ID').mean().astype(int)
# y_unique = y_ID.groupby('customer_ID').agg(['mean', 'last', 'first']).astype(int)


print('done ✅')

print(X_avg_pp.shape , y_unique.shape)


done ✅
(26670, 910) (26670, 1)


# XG Boost & Grid Search

In [8]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold 
import xgboost as xgb
print('XGB Version',xgb.__version__)

print('done ✅')


XGB Version 2.0.0-dev
done ✅


In [ ]:
%%time

xgb_class = XGBClassifier()

# Hyperparameter Grid
grid = {'max_depth': [3, 5, 7], 
        'n_estimators': [80, 90, 100],
        'learning_rate': [0.08, 0.1, 0.12]
         }

# Instanciate Grid Search
search = GridSearchCV(xgb_class, 
                      grid, 
                      scoring = amex_metric_scorer,
                      cv = 5,
                      n_jobs=-1 # paralellize computation
                     ) 


search.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=2
)

print('done ✅')

In [ ]:
search.best_score_


In [ ]:
search.best_params_


In [ ]:
search.best_estimator_


In [9]:
xgb_best_reg = XGBClassifier(max_depth= 5,
                             n_estimators= 100, 
                             learning_rate= 0.08,
#                             tree_method='gpu_hist',
#                              predictor= 'gpu_predictor',
#                              subsample = 0.8,
#                              objective = 'binary.logistic',
#                              random_state = 42
                            )

xgb_best_reg.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), 
              (X_pp_avg_val, y_unique_val)
             ],  
    # stop iterating when eval loss increases 'x' times in a row
    early_stopping_rounds=100
)




[0]	validation_0-logloss:0.64409	validation_1-logloss:0.64491
[1]	validation_0-logloss:0.60178	validation_1-logloss:0.60368
[2]	validation_0-logloss:0.56526	validation_1-logloss:0.56792
[3]	validation_0-logloss:0.53349	validation_1-logloss:0.53710
[4]	validation_0-logloss:0.50524	validation_1-logloss:0.51016
[5]	validation_0-logloss:0.48032	validation_1-logloss:0.48609
[6]	validation_0-logloss:0.45819	validation_1-logloss:0.46499
[7]	validation_0-logloss:0.43836	validation_1-logloss:0.44632
[8]	validation_0-logloss:0.42053	validation_1-logloss:0.42927
[9]	validation_0-logloss:0.40433	validation_1-logloss:0.41401
[10]	validation_0-logloss:0.38960	validation_1-logloss:0.40028
[11]	validation_0-logloss:0.37647	validation_1-logloss:0.38788
[12]	validation_0-logloss:0.36433	validation_1-logloss:0.37675
[13]	validation_0-logloss:0.35315	validation_1-logloss:0.36661
[14]	validation_0-logloss:0.34324	validation_1-logloss:0.35762
[15]	validation_0-logloss:0.33416	validation_1-logloss:0.34938
[1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.08, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=5, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [10]:
y_pred = xgb_best_reg.predict_proba(X_pp_avg_val)[:,1]


In [11]:
amex_metric(y_unique_val, y_pred)

0.7397451358794905

### Best until now

xgb_best_reg = XGBClassifier(max_depth= 5,
                             n_estimators= 100, 
                             learning_rate= 0.08)
                             
early_stopping_rounds=100


Best = 0.7397

# CatBoost

In [12]:
import catboost as ctb


In [ ]:
%%time

model_CBC = ctb.CatBoostClassifier()

# Hyperparameter Grid
grid_cat = {'max_depth': [4, 5, 6, 7], 
        'iterations': [3000, 3500, 4000, 4500],
#         'learning_rate': [0.1, 0.11, 0.12, 0.13],
            'random_state' : [22, 32, 42]
         }

# Instanciate Grid Search
search = GridSearchCV(model_CBC, 
                      grid_cat, 
                      scoring = amex_metric_scorer,
                      cv = 5,
                      n_jobs=-1 # paralellize computation
                     ) 


search.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=3
)

print('done ✅')

In [ ]:
search.best_score_


In [ ]:
search.best_params_


In [ ]:
search.best_estimator_


In [13]:
model_CBC = ctb.CatBoostClassifier(
    iterations=3000, 
       random_state=32,
       max_depth= 6,
#          n_estimators= 100, 
         learning_rate= 0.11 ,
#        task_type = 'GPU',
                                  )


model_CBC.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=100
)




0:	learn: 0.5764926	test: 0.5765355	test1: 0.5764009	best: 0.5764009 (0)	total: 81.1ms	remaining: 4m 3s
1:	learn: 0.4958155	test: 0.4958414	test1: 0.4964290	best: 0.4964290 (1)	total: 90.4ms	remaining: 2m 15s
2:	learn: 0.4396043	test: 0.4396310	test1: 0.4407757	best: 0.4407757 (2)	total: 99.8ms	remaining: 1m 39s
3:	learn: 0.4027818	test: 0.4028171	test1: 0.4054745	best: 0.4054745 (3)	total: 110ms	remaining: 1m 22s
4:	learn: 0.3746333	test: 0.3746783	test1: 0.3775449	best: 0.3775449 (4)	total: 120ms	remaining: 1m 12s
5:	learn: 0.3507438	test: 0.3507969	test1: 0.3540847	best: 0.3540847 (5)	total: 131ms	remaining: 1m 5s
6:	learn: 0.3336018	test: 0.3336569	test1: 0.3372620	best: 0.3372620 (6)	total: 142ms	remaining: 1m
7:	learn: 0.3206406	test: 0.3206964	test1: 0.3249507	best: 0.3249507 (7)	total: 151ms	remaining: 56.5s
8:	learn: 0.3107500	test: 0.3108047	test1: 0.3151200	best: 0.3151200 (8)	total: 160ms	remaining: 53.3s
9:	learn: 0.3022885	test: 0.3023417	test1: 0.3070891	best: 0.3070891 

80:	learn: 0.2134118	test: 0.2134391	test1: 0.2506885	best: 0.2506885 (80)	total: 890ms	remaining: 32.1s
81:	learn: 0.2128479	test: 0.2128753	test1: 0.2505293	best: 0.2505293 (81)	total: 902ms	remaining: 32.1s
82:	learn: 0.2123203	test: 0.2123475	test1: 0.2504694	best: 0.2504694 (82)	total: 912ms	remaining: 32.1s
83:	learn: 0.2116747	test: 0.2117019	test1: 0.2504055	best: 0.2504055 (83)	total: 924ms	remaining: 32.1s
84:	learn: 0.2110855	test: 0.2111128	test1: 0.2502178	best: 0.2502178 (84)	total: 935ms	remaining: 32.1s
85:	learn: 0.2103035	test: 0.2103308	test1: 0.2503788	best: 0.2502178 (84)	total: 945ms	remaining: 32s
86:	learn: 0.2098922	test: 0.2099195	test1: 0.2504533	best: 0.2502178 (84)	total: 957ms	remaining: 32s
87:	learn: 0.2093716	test: 0.2093988	test1: 0.2504240	best: 0.2502178 (84)	total: 967ms	remaining: 32s
88:	learn: 0.2089418	test: 0.2089689	test1: 0.2503580	best: 0.2502178 (84)	total: 979ms	remaining: 32s
89:	learn: 0.2083409	test: 0.2083679	test1: 0.2504200	best: 0.2

169:	learn: 0.1667564	test: 0.1667815	test1: 0.2481304	best: 0.2481304 (169)	total: 1.9s	remaining: 31.6s
170:	learn: 0.1666242	test: 0.1666494	test1: 0.2481204	best: 0.2481204 (170)	total: 1.94s	remaining: 32.2s
171:	learn: 0.1662846	test: 0.1663096	test1: 0.2479980	best: 0.2479980 (171)	total: 1.98s	remaining: 32.5s
172:	learn: 0.1657809	test: 0.1658059	test1: 0.2478974	best: 0.2478974 (172)	total: 2.01s	remaining: 32.9s
173:	learn: 0.1654140	test: 0.1654391	test1: 0.2477994	best: 0.2477994 (173)	total: 2.03s	remaining: 33s
174:	learn: 0.1651192	test: 0.1651443	test1: 0.2477932	best: 0.2477932 (174)	total: 2.04s	remaining: 33s
175:	learn: 0.1647642	test: 0.1647893	test1: 0.2477691	best: 0.2477691 (175)	total: 2.06s	remaining: 33s
176:	learn: 0.1644987	test: 0.1645237	test1: 0.2477014	best: 0.2477014 (176)	total: 2.07s	remaining: 33s
177:	learn: 0.1640243	test: 0.1640494	test1: 0.2476288	best: 0.2476288 (177)	total: 2.08s	remaining: 32.9s
178:	learn: 0.1636821	test: 0.1637072	test1: 0

259:	learn: 0.1336119	test: 0.1336361	test1: 0.2480564	best: 0.2469765 (222)	total: 2.92s	remaining: 30.8s
260:	learn: 0.1333215	test: 0.1333457	test1: 0.2480915	best: 0.2469765 (222)	total: 2.93s	remaining: 30.8s
261:	learn: 0.1329746	test: 0.1329988	test1: 0.2481651	best: 0.2469765 (222)	total: 2.94s	remaining: 30.8s
262:	learn: 0.1326804	test: 0.1327046	test1: 0.2481490	best: 0.2469765 (222)	total: 2.96s	remaining: 30.8s
263:	learn: 0.1323755	test: 0.1323996	test1: 0.2480514	best: 0.2469765 (222)	total: 2.97s	remaining: 30.8s
264:	learn: 0.1320779	test: 0.1321020	test1: 0.2480924	best: 0.2469765 (222)	total: 2.98s	remaining: 30.8s
265:	learn: 0.1319053	test: 0.1319294	test1: 0.2481840	best: 0.2469765 (222)	total: 2.99s	remaining: 30.8s
266:	learn: 0.1317365	test: 0.1317606	test1: 0.2481144	best: 0.2469765 (222)	total: 3.01s	remaining: 30.8s
267:	learn: 0.1314595	test: 0.1314836	test1: 0.2481520	best: 0.2469765 (222)	total: 3.02s	remaining: 30.8s
268:	learn: 0.1311322	test: 0.1311563

In [14]:
y_pred = model_CBC.predict_proba(X_pp_avg_val)[:,1]


In [15]:
amex_metric(y_unique_val, y_pred)

0.7400690509839548

In [51]:
model_CBC = ctb.CatBoostClassifier(iterations=5000, 
                                   random_state=22, 
#                                    task_type = 'GPU'
                                  )

model_CBC.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 100 times in a row
    early_stopping_rounds=100
)




Learning rate set to 0.032421
0:	learn: 0.6590648	test: 0.6591101	test1: 0.6588794	best: 0.6588794 (0)	total: 21.7ms	remaining: 1m 48s
1:	learn: 0.6259540	test: 0.6260125	test1: 0.6259032	best: 0.6259032 (1)	total: 36.2ms	remaining: 1m 30s
2:	learn: 0.5981965	test: 0.5982511	test1: 0.5982448	best: 0.5982448 (2)	total: 48.6ms	remaining: 1m 20s
3:	learn: 0.5712616	test: 0.5713062	test1: 0.5713872	best: 0.5713872 (3)	total: 64.5ms	remaining: 1m 20s
4:	learn: 0.5453963	test: 0.5454303	test1: 0.5453502	best: 0.5453502 (4)	total: 77.6ms	remaining: 1m 17s
5:	learn: 0.5207451	test: 0.5207726	test1: 0.5207040	best: 0.5207040 (5)	total: 97.4ms	remaining: 1m 21s
6:	learn: 0.5028288	test: 0.5028534	test1: 0.5029181	best: 0.5029181 (6)	total: 114ms	remaining: 1m 21s
7:	learn: 0.4856552	test: 0.4856776	test1: 0.4858263	best: 0.4858263 (7)	total: 126ms	remaining: 1m 18s
8:	learn: 0.4691526	test: 0.4691750	test1: 0.4693740	best: 0.4693740 (8)	total: 141ms	remaining: 1m 18s
9:	learn: 0.4534770	test: 0.

86:	learn: 0.2519770	test: 0.2520141	test1: 0.2646227	best: 0.2646227 (86)	total: 1.46s	remaining: 1m 22s
87:	learn: 0.2515772	test: 0.2516141	test1: 0.2643454	best: 0.2643454 (87)	total: 1.48s	remaining: 1m 22s
88:	learn: 0.2511891	test: 0.2512258	test1: 0.2641950	best: 0.2641950 (88)	total: 1.49s	remaining: 1m 22s
89:	learn: 0.2508225	test: 0.2508592	test1: 0.2639164	best: 0.2639164 (89)	total: 1.5s	remaining: 1m 22s
90:	learn: 0.2505398	test: 0.2505763	test1: 0.2638135	best: 0.2638135 (90)	total: 1.52s	remaining: 1m 21s
91:	learn: 0.2502325	test: 0.2502689	test1: 0.2636518	best: 0.2636518 (91)	total: 1.53s	remaining: 1m 21s
92:	learn: 0.2498942	test: 0.2499305	test1: 0.2634757	best: 0.2634757 (92)	total: 1.54s	remaining: 1m 21s
93:	learn: 0.2495164	test: 0.2495526	test1: 0.2633039	best: 0.2633039 (93)	total: 1.55s	remaining: 1m 21s
94:	learn: 0.2490281	test: 0.2490640	test1: 0.2629648	best: 0.2629648 (94)	total: 1.57s	remaining: 1m 20s
95:	learn: 0.2487270	test: 0.2487628	test1: 0.2

164:	learn: 0.2309287	test: 0.2309589	test1: 0.2548429	best: 0.2548429 (164)	total: 2.74s	remaining: 1m 20s
165:	learn: 0.2307533	test: 0.2307836	test1: 0.2547468	best: 0.2547468 (165)	total: 2.75s	remaining: 1m 20s
166:	learn: 0.2305951	test: 0.2306254	test1: 0.2547052	best: 0.2547052 (166)	total: 2.78s	remaining: 1m 20s
167:	learn: 0.2303728	test: 0.2304031	test1: 0.2546582	best: 0.2546582 (167)	total: 2.8s	remaining: 1m 20s
168:	learn: 0.2302112	test: 0.2302416	test1: 0.2546327	best: 0.2546327 (168)	total: 2.81s	remaining: 1m 20s
169:	learn: 0.2300397	test: 0.2300700	test1: 0.2545783	best: 0.2545783 (169)	total: 2.83s	remaining: 1m 20s
170:	learn: 0.2297470	test: 0.2297773	test1: 0.2544948	best: 0.2544948 (170)	total: 2.84s	remaining: 1m 20s
171:	learn: 0.2295861	test: 0.2296163	test1: 0.2544524	best: 0.2544524 (171)	total: 2.85s	remaining: 1m 20s
172:	learn: 0.2294116	test: 0.2294418	test1: 0.2544117	best: 0.2544117 (172)	total: 2.87s	remaining: 1m 19s
173:	learn: 0.2292359	test: 0

240:	learn: 0.2176122	test: 0.2176406	test1: 0.2514892	best: 0.2514375 (238)	total: 3.97s	remaining: 1m 18s
241:	learn: 0.2174716	test: 0.2175000	test1: 0.2514166	best: 0.2514166 (241)	total: 3.99s	remaining: 1m 18s
242:	learn: 0.2172780	test: 0.2173063	test1: 0.2513329	best: 0.2513329 (242)	total: 4.01s	remaining: 1m 18s
243:	learn: 0.2171004	test: 0.2171287	test1: 0.2512883	best: 0.2512883 (243)	total: 4.03s	remaining: 1m 18s
244:	learn: 0.2169328	test: 0.2169610	test1: 0.2511981	best: 0.2511981 (244)	total: 4.04s	remaining: 1m 18s
245:	learn: 0.2168211	test: 0.2168493	test1: 0.2511828	best: 0.2511828 (245)	total: 4.06s	remaining: 1m 18s
246:	learn: 0.2166762	test: 0.2167044	test1: 0.2511240	best: 0.2511240 (246)	total: 4.07s	remaining: 1m 18s
247:	learn: 0.2165066	test: 0.2165348	test1: 0.2511083	best: 0.2511083 (247)	total: 4.09s	remaining: 1m 18s
248:	learn: 0.2162974	test: 0.2163256	test1: 0.2510735	best: 0.2510735 (248)	total: 4.1s	remaining: 1m 18s
249:	learn: 0.2161459	test: 0

325:	learn: 0.2037657	test: 0.2037930	test1: 0.2489538	best: 0.2489538 (325)	total: 5.44s	remaining: 1m 18s
326:	learn: 0.2035602	test: 0.2035875	test1: 0.2489524	best: 0.2489524 (326)	total: 5.46s	remaining: 1m 17s
327:	learn: 0.2033865	test: 0.2034138	test1: 0.2489355	best: 0.2489355 (327)	total: 5.47s	remaining: 1m 17s
328:	learn: 0.2032013	test: 0.2032286	test1: 0.2488874	best: 0.2488874 (328)	total: 5.49s	remaining: 1m 17s
329:	learn: 0.2030462	test: 0.2030734	test1: 0.2488699	best: 0.2488699 (329)	total: 5.54s	remaining: 1m 18s
330:	learn: 0.2028679	test: 0.2028951	test1: 0.2488350	best: 0.2488350 (330)	total: 5.55s	remaining: 1m 18s
331:	learn: 0.2026774	test: 0.2027045	test1: 0.2488242	best: 0.2488242 (331)	total: 5.57s	remaining: 1m 18s
332:	learn: 0.2025301	test: 0.2025573	test1: 0.2488080	best: 0.2488080 (332)	total: 5.58s	remaining: 1m 18s
333:	learn: 0.2023099	test: 0.2023370	test1: 0.2487980	best: 0.2487980 (333)	total: 5.6s	remaining: 1m 18s
334:	learn: 0.2021145	test: 0

...

818:	learn: 0.1380816	test: 0.1381059	test1: 0.2454287	best: 0.2453534 (798)	total: 13s	remaining: 1m 6s
819:	learn: 0.1379985	test: 0.1380227	test1: 0.2454429	best: 0.2453534 (798)	total: 13s	remaining: 1m 6s
820:	learn: 0.1378924	test: 0.1379167	test1: 0.2454390	best: 0.2453534 (798)	total: 13s	remaining: 1m 6s
821:	learn: 0.1377738	test: 0.1377982	test1: 0.2454068	best: 0.2453534 (798)	total: 13s	remaining: 1m 6s
822:	learn: 0.1376680	test: 0.1376923	test1: 0.2454151	best: 0.2453534 (798)	total: 13s	remaining: 1m 6s
823:	learn: 0.1375702	test: 0.1375945	test1: 0.2454063	best: 0.2453534 (798)	total: 13.1s	remaining: 1m 6s
824:	learn: 0.1374750	test: 0.1374993	test1: 0.2453846	best: 0.2453534 (798)	total: 13.1s	remaining: 1m 6s
825:	learn: 0.1373982	test: 0.1374225	test1: 0.2453734	best: 0.2453534 (798)	total: 13.1s	remaining: 1m 6s
826:	learn: 0.1372845	test: 0.1373088	test1: 0.2454024	best: 0.2453534 (798)	total: 13.1s	remaining: 1m 6s
827:	learn: 0.1371986	test: 0.1372230	test1: 0.

903:	learn: 0.1298134	test: 0.1298374	test1: 0.2450927	best: 0.2450795 (898)	total: 14.4s	remaining: 1m 5s
904:	learn: 0.1297164	test: 0.1297404	test1: 0.2450829	best: 0.2450795 (898)	total: 14.4s	remaining: 1m 5s
905:	learn: 0.1296094	test: 0.1296333	test1: 0.2451363	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
906:	learn: 0.1295299	test: 0.1295538	test1: 0.2451465	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
907:	learn: 0.1294446	test: 0.1294685	test1: 0.2451446	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
908:	learn: 0.1293565	test: 0.1293804	test1: 0.2451421	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
909:	learn: 0.1292289	test: 0.1292528	test1: 0.2451701	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
910:	learn: 0.1291321	test: 0.1291561	test1: 0.2451443	best: 0.2450795 (898)	total: 14.5s	remaining: 1m 5s
911:	learn: 0.1290167	test: 0.1290407	test1: 0.2451219	best: 0.2450795 (898)	total: 14.6s	remaining: 1m 5s
912:	learn: 0.1289263	test: 0.1289503

989:	learn: 0.1219669	test: 0.1219904	test1: 0.2449155	best: 0.2448894 (986)	total: 15.9s	remaining: 1m 4s
990:	learn: 0.1219236	test: 0.1219471	test1: 0.2449063	best: 0.2448894 (986)	total: 15.9s	remaining: 1m 4s
991:	learn: 0.1218078	test: 0.1218313	test1: 0.2448900	best: 0.2448894 (986)	total: 15.9s	remaining: 1m 4s
992:	learn: 0.1217303	test: 0.1217538	test1: 0.2449151	best: 0.2448894 (986)	total: 15.9s	remaining: 1m 4s
993:	learn: 0.1216134	test: 0.1216369	test1: 0.2449197	best: 0.2448894 (986)	total: 16s	remaining: 1m 4s
994:	learn: 0.1215241	test: 0.1215477	test1: 0.2449268	best: 0.2448894 (986)	total: 16s	remaining: 1m 4s
995:	learn: 0.1214362	test: 0.1214597	test1: 0.2448935	best: 0.2448894 (986)	total: 16s	remaining: 1m 4s
996:	learn: 0.1213176	test: 0.1213411	test1: 0.2449042	best: 0.2448894 (986)	total: 16s	remaining: 1m 4s
997:	learn: 0.1212300	test: 0.1212534	test1: 0.2449286	best: 0.2448894 (986)	total: 16.1s	remaining: 1m 4s
998:	learn: 0.1211103	test: 0.1211337	test1: 

1078:	learn: 0.1146309	test: 0.1146541	test1: 0.2450606	best: 0.2448730 (1014)	total: 17.3s	remaining: 1m 2s
1079:	learn: 0.1145729	test: 0.1145960	test1: 0.2450477	best: 0.2448730 (1014)	total: 17.3s	remaining: 1m 2s
1080:	learn: 0.1145083	test: 0.1145315	test1: 0.2450502	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1081:	learn: 0.1144245	test: 0.1144477	test1: 0.2450381	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1082:	learn: 0.1143462	test: 0.1143694	test1: 0.2450183	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1083:	learn: 0.1142721	test: 0.1142953	test1: 0.2450011	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1084:	learn: 0.1142068	test: 0.1142299	test1: 0.2449876	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1085:	learn: 0.1141463	test: 0.1141694	test1: 0.2449624	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1086:	learn: 0.1140384	test: 0.1140615	test1: 0.2448909	best: 0.2448730 (1014)	total: 17.4s	remaining: 1m 2s
1087:	learn: 0.1139

1159:	learn: 0.1083150	test: 0.1083381	test1: 0.2448734	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1160:	learn: 0.1082242	test: 0.1082473	test1: 0.2448349	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1161:	learn: 0.1081384	test: 0.1081615	test1: 0.2448661	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1162:	learn: 0.1080841	test: 0.1081072	test1: 0.2448715	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1163:	learn: 0.1080013	test: 0.1080243	test1: 0.2448975	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1164:	learn: 0.1079001	test: 0.1079231	test1: 0.2449070	best: 0.2447495 (1116)	total: 18.6s	remaining: 1m 1s
1165:	learn: 0.1078472	test: 0.1078702	test1: 0.2449030	best: 0.2447495 (1116)	total: 18.7s	remaining: 1m 1s
1166:	learn: 0.1077874	test: 0.1078103	test1: 0.2448893	best: 0.2447495 (1116)	total: 18.7s	remaining: 1m 1s
1167:	learn: 0.1077274	test: 0.1077504	test1: 0.2448576	best: 0.2447495 (1116)	total: 18.7s	remaining: 1m 1s
1168:	learn: 0.1076

In [52]:
y_pred = model_CBC.predict_proba(X_pp_avg_val)[:,1]


In [53]:
amex_metric(y_unique_val, y_pred)

0.7468562780979094

In [39]:
print (X_pp_avg_train.shape, y_unique_train.shape)

(18669, 182) (18669, 1)


In [43]:
import pickle

In [44]:
pickle.dump(model_CBC, open('../pickles/cat_boost_075', 'wb'))

### Best until now

model_CBC = ctb.CatBoostClassifier(iterations=5000, 
                                   random_state=22
                                  )
                             
early_stopping_rounds=100


Best = 0.7539

# Cat-Boost & Optuna

https://optuna.org/

In [19]:
import optuna
from sklearn.metrics import accuracy_score


In [20]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_pp_avg,y_unique, test_size=0.3)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = ctb.CatBoostClassifier(**param)
    gbm = ctb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(valid_y, pred_labels)
    return accuracy

In [21]:
%%time
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=600)

[I 2022-09-06 14:46:48,390] A new study created in memory with name: no-name-9563455e-8766-4b66-a32f-8720fb1bc1d6
[I 2022-09-06 14:47:01,916] Trial 0 finished with value: 0.89501312335958 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08693493234319488, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.2172473720864343}. Best is trial 0 with value: 0.89501312335958.
[I 2022-09-06 14:47:47,910] Trial 1 finished with value: 0.889763779527559 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.04910626041261207, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.322360252204148}. Best is trial 0 with value: 0.89501312335958.
[I 2022-09-06 14:48:00,242] Trial 2 finished with value: 0.8926384201974753 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.012125567445736842, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample

[I 2022-09-06 14:51:47,728] Trial 26 finished with value: 0.8888888888888888 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08489845427320424, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.506966197388123}. Best is trial 0 with value: 0.89501312335958.
[I 2022-09-06 14:51:59,932] Trial 27 finished with value: 0.8932633420822397 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.032668606824735595, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.9596807774139768}. Best is trial 0 with value: 0.89501312335958.
[I 2022-09-06 14:52:12,784] Trial 28 finished with value: 0.8880139982502188 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.03314568977026316, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 2.3123116292387182}. Best is trial 0 with value: 0.89501312335958.
[I 2022-09-06 14:53:02,059] Trial 29 finished 

CPU times: user 29min 25s, sys: 10min 36s, total: 40min 2s
Wall time: 10min 55s


In [22]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 43
Best trial:
  Value: 0.8956380452443444
  Params: 
    objective: Logloss
    colsample_bylevel: 0.0705495263963352
    depth: 10
    boosting_type: Ordered
    bootstrap_type: MVS


In [23]:
param_dist = { }
for key, value in trial.params.items():
    param_dist[key]=value
param_dist


{'objective': 'Logloss',
 'colsample_bylevel': 0.0705495263963352,
 'depth': 10,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS'}

In [48]:
gbm = ctb.CatBoostClassifier(**param_dist,iterations=5000)

model_CBC.fit(X_pp_avg_train, y_unique_train,
    # evaluate loss at each iteration
    eval_set=[(X_pp_avg_train,y_unique_train), (X_pp_avg_val, y_unique_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=100
)


Learning rate set to 0.008218
0:	learn: 0.6843013	total: 20ms	remaining: 1m 39s
1:	learn: 0.6750722	total: 33.2ms	remaining: 1m 22s
2:	learn: 0.6667166	total: 44.6ms	remaining: 1m 14s
3:	learn: 0.6580421	total: 57.2ms	remaining: 1m 11s
4:	learn: 0.6491690	total: 75.8ms	remaining: 1m 15s
5:	learn: 0.6402522	total: 87.1ms	remaining: 1m 12s
6:	learn: 0.6329898	total: 98.5ms	remaining: 1m 10s
7:	learn: 0.6254789	total: 114ms	remaining: 1m 10s
8:	learn: 0.6181434	total: 124ms	remaining: 1m 9s
9:	learn: 0.6096090	total: 137ms	remaining: 1m 8s
10:	learn: 0.6021988	total: 154ms	remaining: 1m 9s
11:	learn: 0.5947460	total: 166ms	remaining: 1m 8s
12:	learn: 0.5877849	total: 177ms	remaining: 1m 7s
13:	learn: 0.5812210	total: 191ms	remaining: 1m 7s
14:	learn: 0.5750302	total: 202ms	remaining: 1m 7s
15:	learn: 0.5679367	total: 219ms	remaining: 1m 8s
16:	learn: 0.5617033	total: 233ms	remaining: 1m 8s
17:	learn: 0.5552729	total: 251ms	remaining: 1m 9s
18:	learn: 0.5495914	total: 263ms	remaining: 1m 9

166:	learn: 0.2872311	total: 2.49s	remaining: 1m 11s
167:	learn: 0.2867497	total: 2.5s	remaining: 1m 11s
168:	learn: 0.2863147	total: 2.51s	remaining: 1m 11s
169:	learn: 0.2859577	total: 2.53s	remaining: 1m 11s
170:	learn: 0.2855244	total: 2.55s	remaining: 1m 12s
171:	learn: 0.2852151	total: 2.57s	remaining: 1m 12s
172:	learn: 0.2848114	total: 2.58s	remaining: 1m 12s
173:	learn: 0.2843309	total: 2.6s	remaining: 1m 12s
174:	learn: 0.2839323	total: 2.61s	remaining: 1m 12s
175:	learn: 0.2835323	total: 2.63s	remaining: 1m 12s
176:	learn: 0.2831755	total: 2.65s	remaining: 1m 12s
177:	learn: 0.2828571	total: 2.66s	remaining: 1m 12s
178:	learn: 0.2824505	total: 2.67s	remaining: 1m 12s
179:	learn: 0.2820267	total: 2.7s	remaining: 1m 12s
180:	learn: 0.2817240	total: 2.71s	remaining: 1m 12s
181:	learn: 0.2813641	total: 2.72s	remaining: 1m 12s
182:	learn: 0.2810688	total: 2.74s	remaining: 1m 12s
183:	learn: 0.2807952	total: 2.75s	remaining: 1m 11s
184:	learn: 0.2804001	total: 2.76s	remaining: 1m 

333:	learn: 0.2522909	total: 4.98s	remaining: 1m 9s
334:	learn: 0.2521909	total: 4.99s	remaining: 1m 9s
335:	learn: 0.2521128	total: 5.01s	remaining: 1m 9s
336:	learn: 0.2519897	total: 5.03s	remaining: 1m 9s
337:	learn: 0.2518905	total: 5.04s	remaining: 1m 9s
338:	learn: 0.2517976	total: 5.06s	remaining: 1m 9s
339:	learn: 0.2517135	total: 5.07s	remaining: 1m 9s
340:	learn: 0.2516319	total: 5.09s	remaining: 1m 9s
341:	learn: 0.2515457	total: 5.1s	remaining: 1m 9s
342:	learn: 0.2514218	total: 5.12s	remaining: 1m 9s
343:	learn: 0.2513208	total: 5.18s	remaining: 1m 10s
344:	learn: 0.2512274	total: 5.2s	remaining: 1m 10s
345:	learn: 0.2511119	total: 5.21s	remaining: 1m 10s
346:	learn: 0.2510349	total: 5.23s	remaining: 1m 10s
347:	learn: 0.2509436	total: 5.25s	remaining: 1m 10s
348:	learn: 0.2508281	total: 5.27s	remaining: 1m 10s
349:	learn: 0.2507518	total: 5.28s	remaining: 1m 10s
350:	learn: 0.2506525	total: 5.29s	remaining: 1m 10s
351:	learn: 0.2505534	total: 5.31s	remaining: 1m 10s
352:	

493:	learn: 0.2392317	total: 7.27s	remaining: 1m 6s
494:	learn: 0.2391680	total: 7.29s	remaining: 1m 6s
495:	learn: 0.2391219	total: 7.3s	remaining: 1m 6s
496:	learn: 0.2390674	total: 7.32s	remaining: 1m 6s
497:	learn: 0.2390087	total: 7.33s	remaining: 1m 6s
498:	learn: 0.2389532	total: 7.35s	remaining: 1m 6s
499:	learn: 0.2388826	total: 7.36s	remaining: 1m 6s
500:	learn: 0.2388020	total: 7.38s	remaining: 1m 6s
501:	learn: 0.2387459	total: 7.39s	remaining: 1m 6s
502:	learn: 0.2386870	total: 7.4s	remaining: 1m 6s
503:	learn: 0.2386325	total: 7.41s	remaining: 1m 6s
504:	learn: 0.2385742	total: 7.43s	remaining: 1m 6s
505:	learn: 0.2385145	total: 7.44s	remaining: 1m 6s
506:	learn: 0.2384418	total: 7.45s	remaining: 1m 6s
507:	learn: 0.2383800	total: 7.46s	remaining: 1m 5s
508:	learn: 0.2383191	total: 7.48s	remaining: 1m 6s
509:	learn: 0.2382680	total: 7.5s	remaining: 1m 6s
510:	learn: 0.2381902	total: 7.51s	remaining: 1m 5s
511:	learn: 0.2381254	total: 7.53s	remaining: 1m 6s
512:	learn: 0.2

662:	learn: 0.2297962	total: 9.54s	remaining: 1m 2s
663:	learn: 0.2297519	total: 9.55s	remaining: 1m 2s
664:	learn: 0.2297049	total: 9.56s	remaining: 1m 2s
665:	learn: 0.2296568	total: 9.57s	remaining: 1m 2s
666:	learn: 0.2296052	total: 9.59s	remaining: 1m 2s
667:	learn: 0.2295611	total: 9.6s	remaining: 1m 2s
668:	learn: 0.2295105	total: 9.62s	remaining: 1m 2s
669:	learn: 0.2294666	total: 9.63s	remaining: 1m 2s
670:	learn: 0.2294288	total: 9.64s	remaining: 1m 2s
671:	learn: 0.2293861	total: 9.66s	remaining: 1m 2s
672:	learn: 0.2293327	total: 9.67s	remaining: 1m 2s
673:	learn: 0.2292845	total: 9.68s	remaining: 1m 2s
674:	learn: 0.2292276	total: 9.7s	remaining: 1m 2s
675:	learn: 0.2291839	total: 9.71s	remaining: 1m 2s
676:	learn: 0.2291373	total: 9.72s	remaining: 1m 2s
677:	learn: 0.2291013	total: 9.74s	remaining: 1m 2s
678:	learn: 0.2290491	total: 9.75s	remaining: 1m 2s
679:	learn: 0.2290055	total: 9.77s	remaining: 1m 2s
680:	learn: 0.2289495	total: 9.79s	remaining: 1m 2s
681:	learn: 0.

...

4289:	learn: 0.1135925	total: 1m 3s	remaining: 10.5s
4290:	learn: 0.1135672	total: 1m 3s	remaining: 10.5s
4291:	learn: 0.1135497	total: 1m 3s	remaining: 10.5s
4292:	learn: 0.1135316	total: 1m 3s	remaining: 10.5s
4293:	learn: 0.1135134	total: 1m 3s	remaining: 10.4s
4294:	learn: 0.1135012	total: 1m 3s	remaining: 10.4s
4295:	learn: 0.1134756	total: 1m 3s	remaining: 10.4s
4296:	learn: 0.1134557	total: 1m 3s	remaining: 10.4s
4297:	learn: 0.1134288	total: 1m 3s	remaining: 10.4s
4298:	learn: 0.1134087	total: 1m 3s	remaining: 10.4s
4299:	learn: 0.1133859	total: 1m 3s	remaining: 10.4s
4300:	learn: 0.1133660	total: 1m 3s	remaining: 10.3s
4301:	learn: 0.1133495	total: 1m 3s	remaining: 10.3s
4302:	learn: 0.1133330	total: 1m 3s	remaining: 10.3s
4303:	learn: 0.1133174	total: 1m 3s	remaining: 10.3s
4304:	learn: 0.1132942	total: 1m 3s	remaining: 10.3s
4305:	learn: 0.1132752	total: 1m 3s	remaining: 10.3s
4306:	learn: 0.1132577	total: 1m 3s	remaining: 10.3s
4307:	learn: 0.1132351	total: 1m 3s	remaining:

4451:	learn: 0.1104289	total: 1m 5s	remaining: 8.12s
4452:	learn: 0.1104149	total: 1m 5s	remaining: 8.1s
4453:	learn: 0.1103995	total: 1m 5s	remaining: 8.09s
4454:	learn: 0.1103708	total: 1m 5s	remaining: 8.07s
4455:	learn: 0.1103501	total: 1m 5s	remaining: 8.05s
4456:	learn: 0.1103244	total: 1m 5s	remaining: 8.04s
4457:	learn: 0.1103130	total: 1m 6s	remaining: 8.03s
4458:	learn: 0.1102996	total: 1m 6s	remaining: 8.01s
4459:	learn: 0.1102859	total: 1m 6s	remaining: 8s
4460:	learn: 0.1102619	total: 1m 6s	remaining: 7.98s
4461:	learn: 0.1102382	total: 1m 6s	remaining: 7.96s
4462:	learn: 0.1102148	total: 1m 6s	remaining: 7.95s
4463:	learn: 0.1102036	total: 1m 6s	remaining: 7.93s
4464:	learn: 0.1101845	total: 1m 6s	remaining: 7.92s
4465:	learn: 0.1101606	total: 1m 6s	remaining: 7.9s
4466:	learn: 0.1101390	total: 1m 6s	remaining: 7.89s
4467:	learn: 0.1101179	total: 1m 6s	remaining: 7.87s
4468:	learn: 0.1100913	total: 1m 6s	remaining: 7.86s
4469:	learn: 0.1100740	total: 1m 6s	remaining: 7.84

4610:	learn: 0.1074039	total: 1m 8s	remaining: 5.76s
4611:	learn: 0.1073793	total: 1m 8s	remaining: 5.75s
4612:	learn: 0.1073622	total: 1m 8s	remaining: 5.73s
4613:	learn: 0.1073464	total: 1m 8s	remaining: 5.72s
4614:	learn: 0.1073197	total: 1m 8s	remaining: 5.7s
4615:	learn: 0.1073003	total: 1m 8s	remaining: 5.69s
4616:	learn: 0.1072804	total: 1m 8s	remaining: 5.67s
4617:	learn: 0.1072580	total: 1m 8s	remaining: 5.66s
4618:	learn: 0.1072384	total: 1m 8s	remaining: 5.64s
4619:	learn: 0.1072286	total: 1m 8s	remaining: 5.63s
4620:	learn: 0.1072138	total: 1m 8s	remaining: 5.61s
4621:	learn: 0.1072049	total: 1m 8s	remaining: 5.6s
4622:	learn: 0.1071770	total: 1m 8s	remaining: 5.58s
4623:	learn: 0.1071683	total: 1m 8s	remaining: 5.57s
4624:	learn: 0.1071514	total: 1m 8s	remaining: 5.55s
4625:	learn: 0.1071293	total: 1m 8s	remaining: 5.54s
4626:	learn: 0.1070997	total: 1m 8s	remaining: 5.52s
4627:	learn: 0.1070779	total: 1m 8s	remaining: 5.51s
4628:	learn: 0.1070638	total: 1m 8s	remaining: 5

4766:	learn: 0.1047382	total: 1m 10s	remaining: 3.45s
4767:	learn: 0.1047139	total: 1m 10s	remaining: 3.43s
4768:	learn: 0.1046988	total: 1m 10s	remaining: 3.42s
4769:	learn: 0.1046813	total: 1m 10s	remaining: 3.4s
4770:	learn: 0.1046578	total: 1m 10s	remaining: 3.39s
4771:	learn: 0.1046343	total: 1m 10s	remaining: 3.37s
4772:	learn: 0.1046207	total: 1m 10s	remaining: 3.36s
4773:	learn: 0.1045947	total: 1m 10s	remaining: 3.35s
4774:	learn: 0.1045712	total: 1m 10s	remaining: 3.33s
4775:	learn: 0.1045595	total: 1m 10s	remaining: 3.32s
4776:	learn: 0.1045462	total: 1m 10s	remaining: 3.3s
4777:	learn: 0.1045306	total: 1m 10s	remaining: 3.29s
4778:	learn: 0.1045098	total: 1m 10s	remaining: 3.27s
4779:	learn: 0.1044893	total: 1m 10s	remaining: 3.26s
4780:	learn: 0.1044715	total: 1m 10s	remaining: 3.24s
4781:	learn: 0.1044584	total: 1m 10s	remaining: 3.23s
4782:	learn: 0.1044332	total: 1m 10s	remaining: 3.21s
4783:	learn: 0.1044150	total: 1m 10s	remaining: 3.2s
4784:	learn: 0.1043917	total: 1

4921:	learn: 0.1019201	total: 1m 13s	remaining: 1.16s
4922:	learn: 0.1019056	total: 1m 13s	remaining: 1.14s
4923:	learn: 0.1018866	total: 1m 13s	remaining: 1.13s
4924:	learn: 0.1018721	total: 1m 13s	remaining: 1.11s
4925:	learn: 0.1018620	total: 1m 13s	remaining: 1.1s
4926:	learn: 0.1018418	total: 1m 13s	remaining: 1.08s
4927:	learn: 0.1018282	total: 1m 13s	remaining: 1.07s
4928:	learn: 0.1018103	total: 1m 13s	remaining: 1.05s
4929:	learn: 0.1017898	total: 1m 13s	remaining: 1.04s
4930:	learn: 0.1017751	total: 1m 13s	remaining: 1.02s
4931:	learn: 0.1017562	total: 1m 13s	remaining: 1.01s
4932:	learn: 0.1017413	total: 1m 13s	remaining: 994ms
4933:	learn: 0.1017186	total: 1m 13s	remaining: 980ms
4934:	learn: 0.1017016	total: 1m 13s	remaining: 965ms
4935:	learn: 0.1016853	total: 1m 13s	remaining: 950ms
4936:	learn: 0.1016671	total: 1m 13s	remaining: 935ms
4937:	learn: 0.1016487	total: 1m 13s	remaining: 921ms
4938:	learn: 0.1016302	total: 1m 13s	remaining: 906ms
4939:	learn: 0.1016174	total:

In [49]:
y_pred = model_CBC.predict_proba(X_pp_avg_val)[:,1]


In [50]:
amex_metric(y_unique_val, y_pred)

0.7539681163513423